### Q1. Refactoring

In [17]:
!python3 batch.py 2023 03

/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Predicted mean duration: 14.203865642696083


### Q2. Installing pytest

In [6]:
!pipenv install --dev pytest

Creating a virtualenv for this project
Pipfile: /Users/kuanysh/Downloads/mlops/mlops-zoomcamp/06-best-practices/Pipfile
Using /opt/homebrew/bin/python33.12.7 to create virtualenv...
⠹ Creating virtual environment...created virtual environment CPython3.12.7.final.0-64 in 191ms
  creator 
CPython3macOsBrew(dest=/Users/kuanysh/.local/share/virtualenvs/06-best-practices
-sBV8QPxH, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, 
app_data_dir=/Users/kuanysh/Library/Application Support/virtualenv)
    added seed packages: pip==24.3.1
  activators 
BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator
,PythonActivator

✔ Successfully created virtual environment!
⠹ Creating virtual environment...
Virtualenv location: /Users/kuanysh/.local/share/virtualenvs/06-best-practices-sBV8QPxH
Pipfile.lock not found, creating...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies....
✔ Suc

In [7]:
!mkdir tests

In [8]:
!touch tests/test_batch.py

In [9]:
!touch tests/__init__.py

In [10]:
ls -lh tests/

total 0
-rw-r--r--@ 1 kuanysh  staff     0B Nov 11 00:44 __init__.py
-rw-r--r--@ 1 kuanysh  staff     0B Nov 11 00:44 test_batch.py


### Q3. Writing first unit test

In [18]:
!python3 unit_test.py

How many rows should be there in the expected dataframe?
2


### Q4. Mocking S3 with Localstack

In [1]:
!docker-compose up -d

WARN[0000] /Users/kuanysh/Downloads/mlops/mlops-zoomcamp/06-best-practices/docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
[+] Running 0/0
 ⠋ Container localstack  Starting                                          0.1s 
[+] Running 1/1
 ✔ Container localstack  Started                                           0.1s 


In [38]:
!aws configure set aws_access_key_id test --profile localstack
!aws configure set aws_secret_access_key test --profile localstack
!aws configure set region us-east-1 --profile localstack

In [34]:
!aws s3 mb s3://nyc-duration --endpoint-url=http://localhost:4566 --profile localstack

make_bucket: nyc-duration


In [35]:
!aws s3 ls --endpoint-url=http://localhost:4566 --profile localstack

2024-11-11 01:19:11 nyc-duration


In [31]:
!cat batch_updated.py

#!/usr/bin/env python
# coding: utf-8

import sys
import pickle
import pandas as pd
import os
from datetime import datetime

INPUT_FILE_PATTERN  = "s3://nyc-duration/in/{year:04d}-{month:02d}.parquet"
OUTPUT_FILE_PATTERN = "s3://nyc-duration/out/{year:04d}-{month:02d}.parquet"
S3_ENDPOINT_URL = os.getenv('S3_ENDPOINT_URL')

def read_data(filename):
    if S3_ENDPOINT_URL:
        storage_options = {
            'client_kwargs': {
                'endpoint_url': S3_ENDPOINT_URL
            }
        }
        df = pd.read_parquet(filename, storage_options=storage_options)
    else:
        df = pd.read_parquet(filename)
    return df

def prepare_data(df):
    df = df.copy()
    df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()
    df['PULocationID'] = df['PULocationID'].fillna(-1).astype(int)
    df['DOLocationID'] = df['DOLocationID'].fillna(-1).astype(int)
    df['PU_D

### Q5. Creating test data

In [43]:
!python3 batch_updated.py 2023 01

/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Predicted mean duration: 23.197149245775073


In [44]:
!aws s3 ls s3://nyc-duration/in/ --endpoint-url=http://localhost:4566 --profile localstack

2024-11-11 02:55:10   47673370 2023-01.parquet


In [45]:
!aws s3 ls s3://nyc-duration/out/ --endpoint-url=http://localhost:4566 --profile localstack

2024-11-11 03:27:05   15248288 2023-01.parquet


In [49]:
!python3 tests/integration_test.py

In [50]:
!aws s3 ls s3://nyc-duration/out/ --endpoint-url=http://localhost:4566 --profile localstack

2024-11-11 03:34:16       4222 2023-01.parquet


### Q6. Finish the integration test

In [51]:
!python3 save_data.py 2023 01

/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Predicted mean duration: 18.138625226015364


In [52]:
!aws s3 ls s3://nyc-duration/out/ --endpoint-url=http://localhost:4566 --profile localstack

2024-11-11 03:49:28       1798 2023-01-save_data.parquet
2024-11-11 03:40:23       4222 2023-01.parquet
